# init

In [ ]:
from huggingface_hub import login

# Replace "YOUR_HF_TOKEN" with your actual token
login()
print("Successfully logged in to Hugging Face!")

In [ ]:
%%capture
import os

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

## load model and tokenizer

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
# from vllm import LLM, S20amplingParams

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

In [ ]:
import torch
import gc
if 'model' in globals():
    del model
if 'tokenizer' in globals():
    del tokenizer

# Dọn dẹp rác bộ nhớ
torch.cuda.empty_cache()
gc.collect()
print("Đã dọn sạch bộ nhớ cũ!")

In [ ]:
max_seq_length = 512
lora_rank = 32 # Larger rank = smarter, but slower
load_in_4bit = True
CHECKPOINT_SOURCE = "/kaggle/input/medical-training/outputs-phase2/checkpoint-500"
custom_model_name ="unsloth/Qwen3-1.7B-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
model_name = CHECKPOINT_SOURCE,
max_seq_length = max_seq_length,   # Context length - can be longer, but uses more memory
load_in_4bit = True,     # 4bit uses much less memory
# load_in_8bit = False,    # A bit more accurate, uses 2x memory
full_finetuning = False, # We have full finetuning now!
device_map = "cuda:0",
# fast_inference=True,
max_lora_rank = lora_rank,
gpu_memory_utilization = 0.95,

)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    lora_dropout = 0.05,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,

)

In [ ]:
from datasets import Dataset
import pandas as pd
import os
import torch
from safetensors.torch import load_file
from peft.utils import set_peft_model_state_dict
print(f"Đang nạp kiến thức từ {CHECKPOINT_SOURCE} vào model mới...")

safetensors_path = os.path.join(CHECKPOINT_SOURCE, "adapter_model.safetensors")

if os.path.exists(safetensors_path):
    adapter_weights = load_file(safetensors_path) # Dùng hàm chuyên dụng
    set_peft_model_state_dict(model, adapter_weights)
    print("Đã nạp kiến thức thành công (Safetensors)!")
else:
    # Fallback nếu file là .bin (PyTorch cũ)
    bin_path = os.path.join(CHECKPOINT_SOURCE, "adapter_model.bin")
    if os.path.exists(bin_path):
        adapter_weights = torch.load(bin_path, map_location="cuda")
        set_peft_model_state_dict(model, adapter_weights)
        print("Đã nạp kiến thức thành công (Bin)!")
    else:
        raise FileNotFoundError(f"Không tìm thấy file adapter_model trong {CHECKPOINT_SOURCE}")

# Load dataset

In [ ]:
# @title 3. Format Dữ liệu CHUYÊN BIỆT (CHỈ EN -> VI)
from datasets import Dataset
import pandas as pd
import os

FINAL_DATA_PATH = "/kaggle/input/transformer-medical-dataprocessing/vlsp_medical_cleaned_final.csv"

# 1. Load dữ liệu
try:
    if os.path.exists(FINAL_DATA_PATH):
        df = pd.read_csv(FINAL_DATA_PATH, dtype={'en': str, 'vi': str})
        full_dataset = Dataset.from_pandas(df)
        print(f"Đã load {len(full_dataset)} dòng dữ liệu gốc.")
    else:
        raise FileNotFoundError(f"Không tìm thấy file tại {FINAL_DATA_PATH}")
except Exception as e:
    print(f"Lỗi: {e}")
    raise

# 2. Định nghĩa Prompt (Chỉ cần 1 cái)
sys_prompt_en_to_vi = (
    "Bạn là một biên dịch viên y tế chuyên nghiệp. "
    "Nhiệm vụ của bạn là dịch chính xác văn bản y khoa từ tiếng Anh sang tiếng Việt, "
    "đảm bảo văn phong khoa học và thuật ngữ chính xác."
)

def formatting_prompts_en_vi_only(examples):
    convos = []
    inputs = examples["en"]
    outputs = examples["vi"]
    
    for en, vi in zip(inputs, outputs):
        # CHỈ GIỮ LẠI CHIỀU EN -> VI
        convos.append([
            {"role": "system", "content": sys_prompt_en_to_vi},
            {"role": "user", "content": str(en)},
            {"role": "assistant", "content": str(vi)},
        ])
        
    
    texts = [tokenizer.apply_chat_template(
            c, 
            tokenize=False, 
            add_generation_prompt=False,
            enable_thinking=False 
        ) for c in convos]
    return { "text" : texts }

# --- ÁP DỤNG ---
print("Đang format dữ liệu chuyên biệt (Chỉ En -> Vi)...")
train_dataset = full_dataset.map(
    formatting_prompts_en_vi_only, # Gọi hàm mới
    batched = True, 
    remove_columns=full_dataset.column_names
)

train_dataset = train_dataset.shuffle(seed=1234)

print("-" * 50)
print(f"Số lượng mẫu huấn luyện (En-Vi Only): {len(train_dataset)}")

In [ ]:
# Kiểm tra xem có bao nhiêu tham số 
model.print_trainable_parameters()

In [ ]:
from trl import SFTTrainer, SFTConfig
import torch

LAST_CHECKPOINT = "/kaggle/input/medical-training/outputs-phase2/checkpoint-2000"


BATCH_SIZE_PER_GPU = 16
GRAD_ACCUM = 4
training_args = SFTConfig(
    output_dir = "outputs_adapter_en_vi",
    dataset_text_field = "text",
    max_seq_length = max_seq_length, 
    
    per_device_train_batch_size = BATCH_SIZE_PER_GPU, 
    gradient_accumulation_steps = GRAD_ACCUM,
    
    warmup_steps = 100,
    num_train_epochs = 1,
    
    # --- ĐIỀU CHỈNH CỤ THỂ CHO GIỚI HẠN 7 GIỜ ---
    max_steps = 1001,
    # ---

    learning_rate = 1e-5,
    lr_scheduler_type = "constant_with_warmup",
    
    fp16 = True, # Qwen3-1.7B chạy FP16 là ổn định nhất
    bf16 = False,
    optim = "adamw_8bit", 
    weight_decay = 0.01,
    
    # Checkpoint
    logging_steps = 10,
    eval_strategy = "no",
    #eval_steps = 250, # Kiểm tra chất lượng và lưu checkpoint mỗi 250 bước
    save_strategy = "steps",
    save_steps = 500, 
    save_total_limit = 3,
    
    seed = 21,
    report_to = "none",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    #eval_dataset = eval_dataset, 
    args = training_args,
)

print(f"Trainer đang hiểu save_steps là: {trainer.args.save_steps}")
# --- 5. START TRAINING ---
trainer.train()

# --- 6. SAVE FINAL ---
OUTPUT_DIR = "Qwen1.7B_Medical_En_Vi_Specialist" # Tên file đích
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Hoàn tất! Adapter En-Vi đã lưu tại {OUTPUT_DIR}")